In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
with open('.secret/yelp_api.json') as f:
    login = json.load(f)
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [3]:
# set our API call parameters 
LOCATION = 'Dallas,TX'
TERM = 'jollof'

In [5]:
#Create a results-in-progress JSON file, but only if it doesn't exist

# Specifying JSON_FILE filename
JSON_FILE = "Data/results_in_progress_Dallas_jollof.json"

## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_Dallas_jollof.json not found. Saving empty list to file.


In [6]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [7]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [8]:
## How many results total?
total_results = results['total']
total_results

34

In [9]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [10]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

2

In [11]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [12]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    # add a 200ms pause
    time.sleep(.2)

  0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,tH3yxO4gKjw6CiR8-s5Zag,jollof-haus-west-african-flavors-allen,Jollof Haus - West African Flavors,https://s3-media4.fl.yelpcdn.com/bphoto/8R8VHd...,False,https://www.yelp.com/biz/jollof-haus-west-afri...,2,"[{'alias': 'african', 'title': 'African'}]",4.5,"{'latitude': 33.12915364706694, 'longitude': -...",[],"{'address1': '945 W Stacy Rd', 'address2': 'St...",19723328185,(972) 332-8185,34330.087885,NaN
1,kLUilzZ0knRWfi2sGPw5wA,fusion-vibes-kitchen-bar-richardson,Fusion Vibes Kitchen + Bar,https://s3-media2.fl.yelpcdn.com/bphoto/8TTdBs...,False,https://www.yelp.com/biz/fusion-vibes-kitchen-...,80,"[{'alias': 'african', 'title': 'African'}, {'a...",4.0,"{'latitude': 32.94914411061217, 'longitude': -...","[pickup, delivery]","{'address1': '100 S Central Expy', 'address2':...",19725433888,(972) 543-3888,13625.870670,NaN
2,fAlx1qzwsrMwERyIHLpi-g,west-african-way-plano,West African Way,https://s3-media4.fl.yelpcdn.com/bphoto/bN6rGC...,False,https://www.yelp.com/biz/west-african-way-plan...,22,"[{'alias': 'african', 'title': 'African'}]",4.5,"{'latitude': 33.01918, 'longitude': -96.68258}",[delivery],"{'address1': '1405 Jupiter Rd', 'address2': 'S...",19724242634,(972) 424-2634,22702.049567,NaN
3,4deZdnP0xsgpEVSsUxTRmA,yemsade-restaurant-grand-prairie,Yemsade Restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/gQ84SI...,False,https://www.yelp.com/biz/yemsade-restaurant-gr...,53,"[{'alias': 'african', 'title': 'African'}]",4.0,"{'latitude': 32.6987457, 'longitude': -97.0153...","[pickup, delivery]","{'address1': '3007 S Carrier Pkwy', 'address2'...",19722642111,(972) 264-2111,25384.301750,$$
4,2TLj2Se-p8yZ6x28sZ3uow,afrika-fusion-dallas-5,Afrika Fusion,https://s3-media1.fl.yelpcdn.com/bphoto/iofVaM...,False,https://www.yelp.com/biz/afrika-fusion-dallas-...,107,"[{'alias': 'african', 'title': 'African'}, {'a...",3.0,"{'latitude': 33.00635, 'longitude': -96.828012}","[pickup, delivery]","{'address1': '18900 Dallas Pkwy', 'address2': ...",19722489112,(972) 248-9112,18793.342915,$$


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
27,KaFNK6FzABlNz4qMz5HXWA,murphys-place-dallas-dallas,Murphy's Place Dallas,,False,https://www.yelp.com/biz/murphys-place-dallas-...,1,[],4.0,"{'latitude': 32.92331, 'longitude': -96.7363}",[],"{'address1': '9410 Walnut St', 'address2': Non...",12145708175,(214) 570-8175,11137.558655,NaN
28,t1OFpyRxngMWGNgI293XsQ,boxochops-modern-african-plano,Boxochops Modern African,https://s3-media1.fl.yelpcdn.com/bphoto/1nGPld...,False,https://www.yelp.com/biz/boxochops-modern-afri...,16,"[{'alias': 'african', 'title': 'African'}]",4.0,"{'latitude': 33.09322, 'longitude': -96.79658}","[pickup, delivery]","{'address1': '8500 Ohio Dr', 'address2': 'Ste ...",14698443412,(469) 844-3412,28326.351939,NaN
29,r-wecKPD4tVkV8A5JA_rvQ,ts-buka-arlington,Ts Buka,https://s3-media3.fl.yelpcdn.com/bphoto/b8XYm-...,False,https://www.yelp.com/biz/ts-buka-arlington?adj...,8,"[{'alias': 'african', 'title': 'African'}]",3.5,"{'latitude': 32.7065663, 'longitude': -97.0992...","[pickup, delivery]","{'address1': '807 E Arkansas Ln', 'address2': ...",16822766400,(682) 276-6400,31562.106949,NaN
30,qVDMYXQlgFeYqCK_7tn9rg,aso-rock-restaurant-and-lounge-dallas,Aso Rock Restaurant and Lounge,https://s3-media3.fl.yelpcdn.com/bphoto/sMDXs-...,False,https://www.yelp.com/biz/aso-rock-restaurant-a...,44,"[{'alias': 'lounges', 'title': 'Lounges'}, {'a...",2.5,"{'latitude': 32.902179, 'longitude': -96.714309}","[pickup, delivery]","{'address1': '9220 Skillman St', 'address2': '...",12143431300,(214) 343-1300,10684.692849,$$
31,kV8L7us7CFBU9mjup2TnKw,spiced-n-grillz-dallas,Spiced N Grillz,,False,https://www.yelp.com/biz/spiced-n-grillz-dalla...,1,"[{'alias': 'fooddeliveryservices', 'title': 'F...",1.0,"{'latitude': 32.64484, 'longitude': -96.94971}","[pickup, delivery]","{'address1': None, 'address2': None, 'address3...",14697596475,(469) 759-6475,26245.094926,NaN


In [15]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

0

In [16]:
## Drop duplicate ids and confirm there are no more duplicates (not needed here just for practice)
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0

In [17]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_Dallas_jollof.csv.gz', compression='gzip',index=False)